# Band Selection For Online Sorting

In [40]:
import os
import time

import numpy as np
from scipy.io import loadmat
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from models.elm import ELM

## Background
In practical industry environment, the data transferring speed cannot maintain an extremely high standard for long time when it comes to the hyperspectral domain. Unlike the commonly used line scanned RGB camera whose image frame is a line contain 3 line of value corresponding to the red, blue and green reflection of object, the line scanned hyperspectral camera generate is able to generate hundreds lines of value in each image frame which reval the chemical characteristic of the object, however, bring burden to not only the Ether Net but also the complaint devices to cope with. In order to avoid this situation, the band selection based on the requirement of the hyperspectral camera is essential. The majority of the current band selection focus only on the accuracy acquired with the selected bands, which is far away from satisfying the demand of our sponsor.

In this part, we mainly focus on three factors of the proposed band selection.
1. The influence caused by different kinds of population are of a world different, some papers in tobacco may not affect the final product, while some plastic in tobacco is surely going to make the final product inedible, so the objective function is redesigned to pay different attention to the mistakes made by our model.
2. When it comes to actually use the selected band with the camera, the MROIs (Multiple Regions of Interest) are needed to be set. However, each single ROI (Region of Interest) will cause some band before and after the region to be invalid, the more ROIs are set the more waste will be. So the encoding method is changed to handle this problem to restrict the total number of ROI.
3. Some prior knowledge may already be observed by human, such as the 吸收峰 on some wavelengths of the material, so the optimization algorithm is modified in order to go beyond the normal version of it.

## Encoding
Traditional method often use binary encoding to represent for each band combination, however, this kind of encoding method is not suitable when
$$[[0, n_{b}], [0, M]] \times n$$
where, the $n_{b}$ represents the band number, $M$ means the maximum of the selected bands number for each window, $n$ is the maximum of the selected window.

## Objective Function
In the objective function of our method, we design it based on 3 rules:
- The smaller the number of the bands the better it will be.(less transmission, less computational cost)
- The total number of the window used is the smaller, the better.(higher frame rate)
- The accuracy of the classification is surely important while the different material should be treated with different attitudes according to the harm they caused.
- ！TODO: Class separability criterion
    - 类间距离
    - 类内聚合度

### load dataset for further operations

In [41]:
data_set_dir = "dataset"

class_names_dic = {'white_foma': 3, 'blue_plastic':  2, 'tobacco':  1,
'transparent_plastic': 4, 'background':  0, 'blue_cigarate':  5, 'red_foma':  6,
'yellow_paper':  7, 'cloth':  8, 'white_paper':  9, 'green_plastic':  10, 'yellow_cigarate':  11,
'feather':  12}

preprocessed_data = loadmat(os.path.join(data_set_dir, 'preprocessed.mat'))
bands = preprocessed_data['bands'].copy()
del preprocessed_data['bands']
preprocessed_data = {k: v for k, v in preprocessed_data.items() if not k.startswith("__")}
data = [(value, class_names_dic[class_name]*np.ones((value.shape[0], ))) for class_name, value in preprocessed_data.items() if (not class_name.startswith("__")) or (class_name == 'bands')]
x = np.concatenate([d[0] for d in data], axis=0)
y = np.concatenate([d[1] for d in data], axis=0)
y = np.asarray(y, dtype=int)
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size=0.1, random_state=12, shuffle=True)
print(f"train_x : {train_x.shape}, train_y: {train_y.shape}"
      f"test_x: {test_x.shape}, test_y: {test_y.shape}")

train_x : (1424, 293), train_y: (1424,)test_x: (12817, 293), test_y: (12817,)


### Objective function design
Based on the rule we proposed above, we designed three objective function and we use the sum of them as the final objective function for the optimizer to optimize.
1. Objective function 1 reflect the total bans number
2. Objective function 2 reflect the total window number
3. Objective function 3 reflect the weighted accuracy based on the selected bands.
4. ==========>！TODO: Class separability criterion!<===========================
    - 类间距离
    - 类内聚合度

In [42]:
from sklearn.metrics import precision_score


class ObjFunc(object):
    def __init__(self, train_x, test_x, train_y, test_y, material_weight=None):
        self.train_x, self.test_x, self.train_y, self.test_y = train_x, test_x, train_y, test_y
        self.cache_space = {}
        self.material_weight = material_weight

    @staticmethod
    def obj_func1(agent):
        """
        Calculate the total number of the bands selected.
        :param agent: search agent
        :return: band number
        """
        agent = agent.ravel()
        res = np.sum(np.round(agent[1::2]))
        return res

    @staticmethod
    def obj_func2(agent):
        """
        Calculate the total number of band windows.

        :param agent: search agent
        :return: window number
        """
        agent = agent.ravel()
        return sum(agent[1::2]>0.5)

    def obj_func3(self, agent):
        """
        Calculate the accuracy based on the selected bands
        :param agent:
        :return:
        """
        index, node_num, hash_value = self.decoder(agent)
        if len(index) == 0:
            return 0.0
        if hash_value in self.cache_space:
            return self.cache_space[hash_value]
        # clf = ELM(input_size=len(index), node_num=node_num, output_num=13)
        clf = DecisionTreeClassifier(max_depth=40, class_weight={0: 100, 1:100})
        train_data, test_data = self.train_x[:, index], self.test_x[:, index]
        clf = clf.fit(train_data, self.train_y)
        pred_y = clf.predict(test_data)
        if self.material_weight is not None:
            precision = precision_score(pred_y, self.test_y, average=None)
            return precision.dot(weighted_array)
        acc = accuracy_score(pred_y, test_y)
        self.cache_space[hash_value] = acc
        return acc

    def obj_func(self, agent):
        band_num = self.obj_func1(agent)
        win_num = self.obj_func2(agent)
        acc = self.obj_func3(agent)
        return band_num/36.0 + win_num/6 + (1 - acc)*10

    @staticmethod
    def decoder(agent):
        agent = agent.ravel()
        node_num = round(agent[-1])
        wavelength_selected = []
        for i in range(0, len(agent)-1, 2):
            if round(agent[i + 1]) == 0:
                continue
            for j in range(round(agent[i + 1])):
                wavelength_selected.append(j + agent[i])
        wavelength_selected.sort()
        wavelength_selected = np.array(wavelength_selected, dtype=int)
        hash_value = wavelength_selected.tobytes()
        return wavelength_selected, node_num, hash_value


In [43]:
test_wolf = np.array([40, 2, 20, 2, 30, 2, 128, 0, 20, 0], dtype=float).reshape((1, -1))

In [44]:
material_weight = {'white_foma': 10, 'blue_plastic':  10, 'tobacco':  50,
'transparent_plastic': 10, 'background':  50, 'blue_cigarate':  10, 'red_foma':  10,
'yellow_paper':  10, 'cloth':  10, 'white_paper':  10, 'green_plastic':  10, 'yellow_cigarate':  10,
'feather':  10}
weighted_array = [(class_names_dic[key], material_weight[key]) for key in class_names_dic.keys()]
weighted_array.sort(key=lambda x_in: x_in[0])
weighted_array = np.array([a[1] for a in weighted_array])
weighted_array = weighted_array / weighted_array.sum()
obj = ObjFunc(train_x, test_x, train_y, test_y, weighted_array)
obj.obj_func3(test_wolf)
obj.obj_func(test_wolf)

4.300369391031852

## Grey Wolf Optimization

In [45]:
from optimizer.grey_wolf import GWO

### Conventional Grey wolf Optimization

Here, we build the conventional Grey Wolf Optimization based on this paper. [ref](https://www.sciencedirect.com/science/article/pii/S0965997813001853)

We add a hash block to the objective function before it start to train the model to get the accuracy. It saves the objective function value and the corresponding hash value of the decoded gene to accelerate the judge progress.

In [46]:
# Fix the random seed
np.random.seed(0)
import random
random.seed(0)
# Create the upper boundary and lower boundary.
max_window_size = 6
bands_num = train_x.shape[1]
upper, lower = [], []
for i in range(max_window_size):
    upper.append(bands_num-36)
    upper.append(36)
    lower.append(0)
    lower.append(0)
upper, lower = np.array([upper]), np.array([lower])

In [47]:
# Run the GWO
gwo = GWO(upper_border=upper, lower_border=lower, judge_func=obj.obj_func, goal=0, num_wolf=50, epochs=200, minimize=True)
gwo.run()

Generation  1 finish! Best value =  5.072439422269916
Generation  2 finish! Best value =  4.3111941345168425
Generation  3 finish! Best value =  4.1659612452412516
Generation  4 finish! Best value =  3.888999461511884
Generation  5 finish! Best value =  3.666209938624938
Generation  6 finish! Best value =  3.666209938624938
Generation  7 finish! Best value =  3.666209938624938
Generation  8 finish! Best value =  3.589579746413535
Generation  9 finish! Best value =  3.5522186542990237
Generation  10 finish! Best value =  3.535417275828273
Generation  11 finish! Best value =  3.535417275828273
Generation  12 finish! Best value =  3.535417275828273
Generation  13 finish! Best value =  3.505087433786407
Generation  14 finish! Best value =  3.505087433786407
Generation  15 finish! Best value =  3.505087433786407
Generation  16 finish! Best value =  3.4705816919600285
Generation  17 finish! Best value =  3.4705816919600285
Generation  18 finish! Best value =  3.4705816919600285
Generation  1

In [48]:
wolf, fitness = gwo.alpha_wolf()

idx, _, _ = obj.decoder(wolf)
bands = bands.ravel()
print("selected bands idx: ", idx)
print("selected wavelength (nm): ", bands[idx])
print("selected band number", int(obj.obj_func1(wolf)))
print("selected window number", int(obj.obj_func2(wolf)))

selected bands idx:  [ 19  93  94 186]
selected wavelength (nm):  [570.089 669.196 670.536 793.75 ]
selected band number 4
selected window number 3


### Spirit Grey Wolf Optimization
In most cases of our optimization tasks, we can get some prior knowledge about the optimum result. However, these prior knowledge maybe vague and not sure to be the best optimum result. Conventional optimizer can hardly take advantages of these prior knowledge. So we modified the grey wolf optimizer to handle this problem.
The detailed information about the spirit grey wolf optimization can be found in this [file](./03_spirit_grey_wolf.ipynb).